<a href="https://colab.research.google.com/github/ayyadevaransh-svg/2025_2026_NASA_SLI_ULTIMATE_CODE_REPOSITORY/blob/main/mpnn/examples/proteinmpnn_in_jax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ProteinMPNN in Jax!

---

fixbb monomer design:
 - `pdb="6MRR" chains="A"`

fixbb homooligomer design:
 - `pdb="5XZK" chains="A,B,C" homooligomer=True`

binder design:
 - `pdb="1SSC" chains="A,B" fix_pos="A"`

---


In [1]:
#@title Install colabdesign
import os
try:
  import colabdesign
except:
  os.system("pip -q install git+https://github.com/sokrypton/ColabDesign.git@v1.1.1")
  os.system("ln -s /usr/local/lib/python3.7/dist-packages/colabdesign colabdesign")

from colabdesign.mpnn import mk_mpnn_model, clear_mem
from colabdesign.shared.protein import pdb_to_string

import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
import pandas as pd
import tqdm.notebook
TQDM_BAR_FORMAT = '{l_bar}{bar}| {n_fmt}/{total_fmt} [elapsed: {elapsed} remaining: {remaining}]'

from google.colab import files
from google.colab import data_table
data_table.enable_dataframe_formatter()

def get_pdb(pdb_code=""):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  elif os.path.isfile(pdb_code):
    return pdb_code
  elif len(pdb_code) == 4:
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb")
    return f"{pdb_code}.pdb"
  else:
    os.system(f"wget -qnc https://alphafold.ebi.ac.uk/files/AF-{pdb_code}-F1-model_v3.pdb")
    return f"AF-{pdb_code}-F1-model_v3.pdb"

In [5]:
%%time
#@title Run ProteinMPNN to design new sequences for given backbone

import warnings, os, re
warnings.simplefilter(action='ignore', category=FutureWarning)

os.system("mkdir -p output")

# USER OPTIONS
#@markdown #### ProteinMPNN options
model_name = "v_48_020" #@param ["v_48_002", "v_48_010", "v_48_020", "v_48_030"]
#@markdown #### Input Options
pdb='AF-P0DOW4-F1-model_v6.pdb' #@param {type:"string"}
#@markdown - leave blank to get an upload prompt
chains = "A" #@param {type:"string"}
homooligomer = False #@param {type:"boolean"}
#@markdown #### Design constraints
fix_pos = "1-360" #@param {type:"string"}
#@markdown - specify which positions to keep fixed in the sequence (example: `1,2-10`)
#@markdown - you can also specify chain specific constraints (example: `A1-10,B1-20`)
#@markdown - you can also specify to fix entire chain(s) (example: `A`)
inverse = False #@param {type:"boolean"}
#@markdown - inverse the `fix_pos` selection (define position to "free" [or design] instead of "fix")
rm_aa = "" #@param {type:"string"}
#@markdown - specify amino acid(s) to exclude (example: `C,A,T`)

#@markdown #### Design Options
num_seqs = 32 #@param ["32", "64", "128", "256", "512", "1024"] {type:"raw"}
sampling_temp = 0.1 #@param ["0.0001", "0.1", "0.15", "0.2", "0.25", "0.3", "0.5", "1.0"] {type:"raw"}
#@markdown - Sampling temperature for amino acids, T=0.0 means taking argmax, T>>1.0 means sample randomly.

#@markdown Note: designed sequences are saved to `design.fasta`

# cleaning user options
chains = re.sub("[^A-Za-z]+",",", chains)
if fix_pos == "": fix_pos = None
rm_aa = ",".join(list(re.sub("[^A-Z]+","",rm_aa.upper())))
if rm_aa == "": rm_aa = None

pdb_path = get_pdb(pdb)
if "mpnn_model" not in dir() or model_name_ != model_name:
  mpnn_model = mk_mpnn_model(model_name)
  model_name_ = model_name

mpnn_model.prep_inputs(pdb_filename=pdb_path,
                       chain=chains, homooligomer=homooligomer,
                       fix_pos=fix_pos, inverse=inverse,
                       rm_aa=rm_aa, verbose=True)

#added By 3301Time
import inspect

print("sample signature:", inspect.signature(mpnn_model.sample))
print("prep_inputs signature:", inspect.signature(mpnn_model.prep_inputs))

# show likely places bias could live
print("attrs containing 'bias':", [a for a in dir(mpnn_model) if "bias" in a.lower()])

# check if the model stores inputs in a dict (common)
if hasattr(mpnn_model, "_inputs"):
    print("_inputs keys:", list(mpnn_model._inputs.keys()))
else:
    print("No mpnn_model._inputs found")

# check alphabet / AA order
for name in ["alphabet","AA","aas","aa_order","restypes"]:
    if hasattr(mpnn_model, name):
        print(name, "=", getattr(mpnn_model, name))

import numpy as np
import jax.numpy as jnp

# 1) Grab existing bias array (correct shape)
bias = np.array(mpnn_model._inputs["bias"])   # shape = (L, A)
L, A = bias.shape
print("bias shape:", bias.shape)

# 2) Define the amino-acid order used by ProteinMPNN (standard)
# ProteinMPNN uses: A C D E F G H I K L M N P Q R S T V W Y
aa_order = "ACDEFGHIKLMNPQRSTVWY"

# If A != 20, we will only bias the first 20 columns (safe default)
use_A = min(A, 20)
aa_to_i = {aa:i for i, aa in enumerate(aa_order[:use_A])}

# 3) Apply bias ONLY to the tail (361–445 => 0-index 360–444)
start, end = 360, 445  # python slice end is exclusive

# Reset any old bias in that region (optional but recommended)
bias[start:end, :use_A] = 0.0

# Mild first-pass bias: reduce stickiness (K/R/H), slightly favor D/E and disorder-friendly residues
bias_dict = {
    "K": -0.8, "R": -0.8, "H": -0.2,
    "D": +0.3, "E": +0.3,
    "S": +0.1, "T": +0.1, "Q": +0.1, "N": +0.1, "G": +0.1, "P": +0.1, "A": +0.05,
    "F": -0.2, "W": -0.2, "Y": -0.2,
    "C": -0.3, "M": -0.2,
}

for aa, w in bias_dict.items():
    if aa in aa_to_i:
        bias[start:end, aa_to_i[aa]] = w

# 4) Write bias back into the model inputs
mpnn_model._inputs["bias"] = jnp.array(bias)
print("Bias applied to residues 361–445.")
#----------------------------------------------------------------


out = mpnn_model.sample(num=num_seqs//32, batch=32,
                        temperature=sampling_temp,
                        rescore=homooligomer)

# Added By 3301Time
def tail_stats(seq):
    tail = seq[360:445]  # residues 361-445
    KR = tail.count("K") + tail.count("R")
    DE = tail.count("D") + tail.count("E")
    H  = tail.count("H")
    net = (tail.count("K") + tail.count("R") + 0.1*H) - DE
    return KR, DE, round(net, 1)

# out["seq"] holds the sequences in this notebook
for i in range(min(10, len(out["seq"]))):
    KR, DE, net = tail_stats(out["seq"][i])
    print(i, "tail K+R:", KR, "tail D+E:", DE, "tail net~:", net)
#---------------------------------------------------------------


with open("design.fasta","w") as fasta:
  for n in range(num_seqs):
    line = f'>score:{out["score"][n]:.3f}_seqid:{out["seqid"][n]:.3f}\n{out["seq"][n]}'
    fasta.write(line+"\n")

labels = ["score","seqid","seq"]
data = [[out[k][n] for k in labels] for n in range(num_seqs)]

df = pd.DataFrame(data, columns=labels)
df.to_csv('output/mpnn_results.csv')
data_table.DataTable(df.round(3))

lengths [445]
the following positions will be fixed:
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 23

,score,seqid,seq
0,1.461,0.153,MASTHQSSTEPSSTGKSEETKKDASQGSGQDSKNVTVTKGTGSSAT...
1,1.400,0.259,MASTHQSSTEPSSTGKSEETKKDASQGSGQDSKNVTVTKGTGSSAT...
2,1.410,0.247,MASTHQSSTEPSSTGKSEETKKDASQGSGQDSKNVTVTKGTGSSAT...
3,1.532,0.200,MASTHQSSTEPSSTGKSEETKKDASQGSGQDSKNVTVTKGTGSSAT...
4,1.568,0.235,MASTHQSSTEPSSTGKSEETKKDASQGSGQDSKNVTVTKGTGSSAT...
5,1.419,0.141,MASTHQSSTEPSSTGKSEETKKDASQGSGQDSKNVTVTKGTGSSAT...
6,1.493,0.212,MASTHQSSTEPSSTGKSEETKKDASQGSGQDSKNVTVTKGTGSSAT...
7,1.488,0.224,MASTHQSSTEPSSTGKSEETKKDASQGSGQDSKNVTVTKGTGSSAT...
8,1.427,0.153,MASTHQSSTEPSSTGKSEETKKDASQGSGQDSKNVTVTKGTGSSAT...
9,1.496,0.212,MASTHQSSTEPSSTGKSEETKKDASQGSGQDSKNVTVTKGTGSSAT...


In [ ]:
#@title ### Get amino acid probabilties from ProteinMPNN (optional)
mode = "unconditional" #@param ["unconditional", "conditional", "conditional_fix_pos"]
#@markdown - `unconditional` - P(sequence | structure)
#@markdown - `conditional` - P(sequence | structure, sequence)
#@markdown - `conditional_fix_pos` - P(sequence[not_fixed] | structure, sequence[fix_pos])
show = "all"
import plotly.express as px
from scipy.special import softmax
from colabdesign.mpnn.model import residue_constants
L = sum(mpnn_model._lengths)
fix_pos = mpnn_model._inputs.get("fix_pos",[])
free_pos = np.delete(np.arange(L),fix_pos)

if mode == "conditional":
  ar_mask = 1-np.eye(L)
  logits = mpnn_model.score(ar_mask=ar_mask)["logits"]
  pdb_labels = None
elif mode == "conditional_fix_pos":
  assert "fix_pos" in mpnn_model._inputs, "no positions fixed"
  ar_mask = 1-np.eye(L)
  p = np.delete(np.arange(L),mpnn_model._inputs["fix_pos"])
  ar_mask[free_pos[:,None],free_pos[None,:]] = 0
  logits = mpnn_model.score(ar_mask=ar_mask)["logits"]
  logits = logits[free_pos]
  pdb_labels = np.array([f"{i}_{c}" for c,i in zip(mpnn_model.pdb["idx"]["chain"], mpnn_model.pdb["idx"]["residue"])])
  pdb_labels = pdb_labels[free_pos]
else:
  ar_mask = np.zeros((L,L))
  logits = mpnn_model.score(ar_mask=ar_mask)["logits"]
  pdb_labels = None

pssm = softmax(logits,-1)
np.savetxt("output/pssm.txt",pssm)

fig = px.imshow(np.array(pssm).T,
               labels=dict(x="positions", y="amino acids", color="probability"),
               y=residue_constants.restypes + ["X"],
               x=pdb_labels,
               zmin=0,
               zmax=1,
               template="simple_white",
              )
fig.update_xaxes(side="top")
fig.show()

In [ ]:
#@title Run AlphaFold Prediction on ProteinMPNN sequences (optional)
#@markdown ###AlphaFold Options
num_models = 1 #@param ["1","2","3","4","5"] {type:"raw"}
num_recycles = 1 #@param ["0","1","2","3"] {type:"raw"}
use_multimer = False #@param {type:"boolean"}
use_templates = False #@param {type:"boolean"}
rm_template_interchain = False #@param {type:"boolean"}
if not os.path.isdir("params"):
  os.system("mkdir params")
  os.system("apt-get install aria2 -qq")
  os.system("aria2c -q -x 16 https://storage.googleapis.com/alphafold/alphafold_params_2022-12-06.tar")
  os.system("tar -xf alphafold_params_2022-12-06.tar -C params")

# where pdb files will be save:
if not os.path.isdir("output/all_pdb"):
  os.system("mkdir output/all_pdb")
else:
  os.system("rm output/all_pdb/*")

from colabdesign.af import mk_af_model
af_args = [pdb_path, chains, homooligomer,
           use_multimer, use_templates]
if "af_arg_current" not in dir() or af_args != af_arg_current:
  af_model = mk_af_model(use_multimer=use_multimer,
                         use_templates=use_templates,
                         best_metric="dgram_cce")
  af_model.prep_inputs(pdb_path,chains,homooligomer=homooligomer)
  af_arg_current = [x for x in af_args]

af_model.restart()
af_model.set_opt("template", rm_ic=rm_template_interchain)

with tqdm.notebook.tqdm(total=out["S"].shape[0], bar_format=TQDM_BAR_FORMAT) as pbar:
  for n,S in enumerate(out["S"]):
    seq = S[:af_model._len].argmax(-1)
    af_model.predict(seq=seq,
                    num_recycles=num_recycles,
                    num_models=num_models,
                    verbose=False)
    (rmsd, ptm, plddt) = (af_model.aux["log"][k] for k in ["rmsd","ptm","plddt"])
    af_model.aux["log"]["composite"] = ptm * plddt
    af_model._save_results(save_best=True, verbose=False)
    af_model.save_current_pdb(f"output/all_pdb/n{n}.pdb")
    af_model._k += 1
    pbar.update(1)

af_model.save_pdb(f"output/best.pdb")

data = []
labels = ["dgram_cce","plddt","ptm","i_ptm","rmsd","composite","mpnn","seqid","seq"]
for n,af in enumerate(af_model._tmp["log"]):
  data.append([af["dgram_cce"],
               af["plddt"],
               af["ptm"],
               af["i_ptm"],
               af["rmsd"],
               af["composite"],
               out["score"][n],
               out["seqid"][n],
               out["seq"][n]])

df = pd.DataFrame(data, columns=labels)
df.to_csv('output/alphafold_results.csv')
data_table.DataTable(df.sort_values("dgram_cce").round(3))
#@markdown Note: designed pdbs are saved to `output/all_pdb/`

In [ ]:
#@title download predictions (optional)
from google.colab import files
os.system(f"zip -r output.zip output/")
files.download(f'output.zip')

In [ ]:
#@title display protein (optional) {run: "auto"}
show_best = True #@param {type:"boolean"}
show_idx = 0 #@param {type:"integer"}
#@markdown - Enter index of protein to show, if `show_best` is disabled.
#@markdown - Note: these are NOT sorted and correspond to
#@markdown the index in pandas dataframe above.
color = "pLDDT" #@param ["chain", "pLDDT", "rainbow"]
show_sidechains = False #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}
color_HP = False #@param {type:"boolean"}
animate = True #@param {type:"boolean"}
#@markdown - if `num_models` > 1, will iterate through the models when `animate` is enabled.
if not show_best:
  pdb_str = pdb_to_string(f"output/all_pdb/n{show_idx}.pdb")
else:
  pdb_str = None
af_model.plot_pdb(show_sidechains=show_sidechains,
                  show_mainchains=show_mainchains,
                  color=color, color_HP=color_HP,
                  animate=animate, pdb_str=pdb_str)

In [ ]:
#@title animate (optional)
#@markdown Note: animation frames are sorted worst to best design
def sort_traj(self, metric="dgram_cce"):
  if metric in ["plddt","ptm","i_ptm","seqid","composite"]:
    metric_higher_better = True
  else:
    metric_higher_better = False
  num = len(self._tmp["traj"]["seq"])
  log = self._tmp["log"][-num:]
  if metric in log[0]:
    n = np.array([x[metric] for x in log]).argsort()
    if metric_higher_better: n = n[::-1]
    sub_traj = {k:[v[m] for m in n] for k,v in self._tmp["traj"].items()}
    return sub_traj
  else:
    return None

sub_traj= sort_traj(af_model)

color_by = "plddt" #@param ["chain", "plddt", "rainbow"]
dpi = 100 #@param {type:"integer"}
HTML(af_model.animate(traj={k:v[::-1] for k,v in sub_traj.items()}, color_by=color_by, dpi=dpi))
